<a href="https://colab.research.google.com/github/IgnaBascu/WebScraping-LuffyToys/blob/main/WebScrapping_LuffyToys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Web Scraping de artículos y precios de una página web
--- 
Proyecto para extraer nombres y precios de figuras de colección de la página web www.luffytoys.cl con el propósito de aprender y demostrar el uso de las librerías ***Request*** y ***Beautiful Soup*** en Python. Además de librerías de visualización de datos como ***Pandas***.


In [2]:
# Importar librerías necesarias para Web Scraping y Visualizar datos
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [ ]:
# Guardar el link que será nuestra fuente de datos (Todos los artículos) en una variable y crear otra variable donde guardaremos la respuesta de la llamada HTTP que hará la librería "request" al sitio web elegido
url = "https://luffytoys.cl/collections/figuras-de-coleccion?page=1"
response = requests.get(url)

# Crear una condicional para ver si el sitio nos responde correctamente (Código 200) o nos entrega un código de error
status_code = response.status_code

if status_code == 200:
  print("Link activo :3")
else:
  print(f"Código de error: {status_code}")

Link activo :3


In [ ]:
# Guardamos en una variable el contenido HTML de la respuesta y probamos si nos entrega el código fuente
# Ocupamos la libreria Beautiful Soup para formatear el resultado de html y parseamos el contenido con "lxml" , con ello obtendremos el código HTML para su análisis

html = response.content
soup = bs(html, "lxml")



Ahora con el código de la página, podemos investigar que etiquetas, id y clases existen y cuales nos interesa extraer.

 En este caso, nos interesa saber el nombre y precio de los artículos. Analizando la página web, encontramos que los nombres de los artículos están bajo la clase "**title**" y su precio con la clase "**money**", la cual apuntaremos con la librería soup.


In [ ]:
# Guardamos en una variable el "contenedor" que alberga los artículos (Como argumentos se coloca la etiqueta y su clase/id)
box = soup.find('div',class_="product-details")

# Especificamos más y buscamos los argumentos que tengan el nombre y precio buscado dentro de la variable del "contenedor"
# get_text() nos ayuda a imprimir solo los elementos de parrafo y strip() elimina espacios de los lados del texto

nombres = box.find('span', class_='title').get_text().strip()
precios = box.find('span', class_='money').get_text().strip()
print ([nombres,precios])



['Dragon Ball Z Son Goku -Saiyan Raised on Earth- S.H.Figuarts', '$54.990']


Ahora con lo básico ya listo y aprendido. Iremos a extraer información de todas las páginas de la sección, modificando el código que hemos visto.

In [3]:
# Ocuparemos ciclos for y el comando findAll() para abarcar todos los elementos deseados
# Generamos algoritmo para buscar en múltiples páginas (Podemos usar el número que nos da la web o el código de respuesta 404, depende de como este construida)
# Creamos una lista para almacenar los nombres y sus precios

lista_productos = []

# Creamos algortimo uniendo todo

page = 1

while page != 47:
  url_2 = f"https://luffytoys.cl/collections/figuras-de-coleccion?page={page}"
  response_2 = requests.get(url_2)
  html_2 = response_2.content
  soup_2 = bs(html_2, "lxml")  
  page = page + 1
  
  # Buscamos todos los "contenedores" de los productos a buscar
  productos = soup_2.find_all('div',class_="product-details")

  # Creamos un ciclo for para almacenar los nombres y precios en formato diccionario, en la lista recién creada
  # Ponemos un try a precios2 ya que en la web existen placeholder que ocupan la misma estructura pero no las clases, lo cual nos da error

  for i in productos:
    nombres2 = i.find('span', class_='title').get_text().strip()
    try:
      precios2 = i.find('span', class_='money').get_text().strip()
    except AttributeError:
      precios2 = "No Precio"

    new_producto = {'Nombre': nombres2,'Precio':precios2}
    lista_productos.append(new_producto)

# Removemos los resultados que no queremos en la lista y probamos con print

for j in lista_productos:
  if j["Precio"] == 'No Precio':
    lista_productos.remove(j)
  else:
    print(j)

print(f"\nEl número de productos agregados es de {len(lista_productos)}")




{'Nombre': 'Dragon Ball Z Son Goku -Saiyan Raised on Earth- S.H.Figuarts', 'Precio': '$54.990'}
{'Nombre': 'Dragon Ball GT SUPER SAIYAN 4 SON GOKU S.H.Figuarts', 'Precio': '$69.990'}
{'Nombre': 'Dragon Ball Z Super Saiyan Full Power Son Goku S.H.Figuarts', 'Precio': '$54.990'}
{'Nombre': 'Saint Seiya Alpha Dubhe Siegfried Myth Cloth Ex Hk', 'Precio': '$174.990'}
{'Nombre': 'Dragon Ball Super: Broly - Super Saiyan BLUE Son Goku (Re Stock) S.H.Figuarts', 'Precio': '$74.990'}
{'Nombre': 'Dragon Ball Z First Form Frieza & Pod set  S.H.Figuarts', 'Precio': '$94.990'}
{'Nombre': 'Dragon Ball Z Frieza 4th Form S.H.Figuarts de Bandai', 'Precio': '$59.990'}
{'Nombre': 'Dragon Ball Super Super Hero - Son Goku - Super Hero Ver.  S.H.Figuarts', 'Precio': '$39.990'}
{'Nombre': 'Dragon Ball Z Final Form Cell Dragon Stars', 'Precio': '$29.990'}
{'Nombre': 'Dragon Ball Super: Broly Super Saiyan Broly Full Power (Re Stock) S.H.Figuarts', 'Precio': '$99.990'}
{'Nombre': 'SAINT SEIYA MYTH CLOTH EX - SIRE

Ahora que tenemos los artículos y sus precios, podemos visualizarlos mediante un dataframe de pandas.

In [4]:
df = pd.DataFrame(lista_productos)
df

,Nombre,Precio
0,Dragon Ball Z Son Goku -Saiyan Raised on Earth...,$54.990
1,Dragon Ball GT SUPER SAIYAN 4 SON GOKU S.H.Fig...,$69.990
2,Dragon Ball Z Super Saiyan Full Power Son Goku...,$54.990
3,Saint Seiya Alpha Dubhe Siegfried Myth Cloth E...,$174.990
4,Dragon Ball Super: Broly - Super Saiyan BLUE S...,$74.990
...,...,...
1040,Back Arrow Atlee Ariel Figuarts Mini,$24.990
1041,[Preventa] Boku No Hero Academia DABI POPUP Pa...,$61.990
1042,Captain Marvel DX Ver. Nendoroid,$75.990
1043,Transformers OPTIMUS PRIME Nendoroid,$59.990


O convertir los datos en un archivo excel para su manipulación a partir del dataframe de pandas.

In [5]:
# Creamos el archivo con el metodo with y ExcelWriter
# Se crea un excel en este GoogleColab. Para descargarlo se debe 

with pd.ExcelWriter("Lista_Productos_LuffyToys.xlsx") as engine:
  df.to_excel(excel_writer=engine,sheet_name="Lista Productos")
